In [ ]:
import os
import zipfile

123


In [ ]:
def unzip_data(zip_path, target_path):
    os.makedirs(target_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_path)

# 替换为相应的路径
train_gt_zip_path = "data/data336725/ch8_training_word_gt_v2.zip"
train_images_zip_prefix = "data/data336725/ch8_training_word_images_gt_part_"
eval_gt_zip_path = "data/data336725/ch8_validation_word_gt_v2.zip"
eval_images_zip_path = "data/data336725/ch8_validation_word_images_gt.zip"

train_target_path = "data/train"
eval_target_path = "data/eval"

# 解压训练集标注
unzip_data(train_gt_zip_path, train_target_path)

# 解压训练集图片（多个分包）
for i in range(1, 3):
    zip_path = f"{train_images_zip_prefix}{i}.zip"
    unzip_data(zip_path, train_target_path)

# 解压验证集标注和图片
unzip_data(eval_gt_zip_path, eval_target_path)
unzip_data(eval_images_zip_path, eval_target_path)

In [ ]:
train_parameters = {
    "batch_size":16,
    "epochs":100
}